In [2]:
import serial
import time
import os
import numpy as np

In [ ]:
# SETTINGS
PORT  = '/dev/cu.usbmodem101'
BAUD = 115200
SAVE_DIR = '/6x6/testing_set_2'
os.makedirs(SAVE_DIR, exist_ok=True)

def send_label(ser, x, y):
    label_cmd = f"L{x},{y}\n".encode()
    ser.write(label_cmd)
    print(f"Label sent: ({x},{y})")

def request_matrix(ser):
    ser.write(b'S\n')
    label = None
    matrix = []

    while True:
        line = ser.readline().decode().strip()
        if line.startswith("LABEL:"):
            label = tuple(map(int, line.split(":")[1].split(",")))
        elif line == "END":
            break
        elif line and all(c in '0123456789,-' for c in line):
            row = list(map(int, line.split(',')))
            matrix.append(row)

    return label, np.array(matrix)

def save_matrix(label, matrix, index):
    filename = f"{label[0]}_{label[1]}_{index}.npy"
    filepath = os.path.join(SAVE_DIR, filename)
    np.save(filepath, matrix)
    print(f"Saved: {filepath}")

In [480]:
print("Connecting to Arduino...")
ser = serial.Serial(PORT, BAUD, timeout=1)
time.sleep(2)
print("Connected.")

x_label = 5
y_label = 5
num_samples = 1

send_label(ser, x_label, y_label)

for i in range(num_samples):
    time.sleep(.5)
    label, matrix = request_matrix(ser)
    save_matrix(label, matrix, i)
    time.sleep(.5)

ser.close()
print("END")


Connecting to Arduino...
Connected.
Label sent: (5,5)
Saved: testing_set_2/5_5_0.npy
END


In [ ]:
x = 0
y = 3
sample = 29
matrix = np.load(f'/6x6/training_set_3/{x}_{y}_{sample}.npy')

print(f"Loaded matrix ({x},{y}) sample {sample}:")
print(matrix)

Loaded matrix (0,3) sample 29:
[[2900 2917 2908 2908 2853 3036]
 [3072 3092 3093 3093 3026 3284]
 [2902 2988 2992 2984 2904 3328]
 [2481 2534 2543 2536 2448 2640]
 [2353 2408 2407 2406 2320 2510]
 [2472 2506 2560 2531 2421 2602]]


In [7]:
# SETTINGS
PORT = '/dev/cu.usbmodem101'
BAUD = 115200
SAVE_DIR = 'no_touch_set'
os.makedirs(SAVE_DIR, exist_ok=True)

def send_dummy_label(ser):
    dummy_cmd = "L0,0\n".encode()
    ser.write(dummy_cmd)
    print("Dummy label sent (0,0)")

def request_matrix(ser):
    ser.write(b'S\n')
    matrix = []

    while True:
        line = ser.readline().decode().strip()
        if line == "END":
            break
        elif line and all(c in '0123456789,-' for c in line):
            row = list(map(int, line.split(',')))
            matrix.append(row)

    return np.array(matrix)

def save_no_touch_matrix(matrix, index):
    filename = f"no_touch_{index}.npy"
    filepath = os.path.join(SAVE_DIR, filename)
    np.save(filepath, matrix)
    print(f"Saved: {filepath}")

# Begin serial connection
print("Connecting to Arduino...")
ser = serial.Serial(PORT, BAUD, timeout=1)
time.sleep(2)
print("Connected.")

num_samples = 240  # or however many you want

for i in range(121,num_samples+1):
    send_dummy_label(ser)         # keep Arduino happy
    time.sleep(0.5)               # give time for it to receive label
    matrix = request_matrix(ser)  # collect voltage readings
    save_no_touch_matrix(matrix, i)
    time.sleep(1.0)               # spacing between samples

ser.close()
print("END")

Connecting to Arduino...
Connected.
Dummy label sent (0,0)
Saved: no_touch_set/no_touch_121.npy
Dummy label sent (0,0)
Saved: no_touch_set/no_touch_122.npy
Dummy label sent (0,0)
Saved: no_touch_set/no_touch_123.npy
Dummy label sent (0,0)
Saved: no_touch_set/no_touch_124.npy
Dummy label sent (0,0)
Saved: no_touch_set/no_touch_125.npy
Dummy label sent (0,0)
Saved: no_touch_set/no_touch_126.npy
Dummy label sent (0,0)
Saved: no_touch_set/no_touch_127.npy
Dummy label sent (0,0)
Saved: no_touch_set/no_touch_128.npy
Dummy label sent (0,0)
Saved: no_touch_set/no_touch_129.npy
Dummy label sent (0,0)
Saved: no_touch_set/no_touch_130.npy
Dummy label sent (0,0)
Saved: no_touch_set/no_touch_131.npy
Dummy label sent (0,0)
Saved: no_touch_set/no_touch_132.npy
Dummy label sent (0,0)
Saved: no_touch_set/no_touch_133.npy
Dummy label sent (0,0)
Saved: no_touch_set/no_touch_134.npy
Dummy label sent (0,0)
Saved: no_touch_set/no_touch_135.npy
Dummy label sent (0,0)
Saved: no_touch_set/no_touch_136.npy
Dumm

In [6]:
print(np.load("no_touch_set/no_touch_3.npy"))

[[2194 2268 2257 2256 2240 2207]
 [2309 2376 2361 2359 2334 2281]
 [2205 2280 2276 2272 2257 2216]
 [2138 2202 2192 2190 2183 2134]
 [2079 2158 2153 2146 2147 2077]
 [2135 2178 2214 2190 2210 2073]]
